# Process NRCS Additional Sites

### Prepare Workspace

In [195]:
# Import system libraries
import os
import sys
import warnings
warnings.filterwarnings("ignore")

# Import data manipulation librariaes
import datetime
import pandas as pd
import numpy as np

# Import geospatial libraries
from shapely.geometry import Point, Polygon
import geopandas as gpd
import rioxarray

# Import API libraries
import requests

# Import progress bar
from tqdm import tqdm

# Set working directory
os.chdir('/Users/jessicarapson/Documents/GitHub/water-supply-forecast')

### Load Data from Files

In [191]:
# Load original sites
mflow_orig = pd.read_csv('assets/data/train_monthly_naturalized_flow.csv')
metad_orig = pd.read_csv('assets/data/metadata.csv')

# Load additional sites
mflow_ncrs = pd.read_csv('assets/data/supplementary_nrcs_train_monthly_naturalized_flow.csv')
metad_ncrs = pd.read_csv('assets/data/supplementary_nrcs_metadata.csv')

# Transform data for combination
mflow_ncrs = mflow_ncrs.rename(columns={'nrcs_id': 'site_id'})
metad_ncrs = metad_ncrs.rename(columns={'nrcs_id': 'site_id'})
metad_ncrs['season_start_month'] = 4
metad_ncrs['season_end_month'] = 7
metad_orig = metad_orig[metad_ncrs.columns]

# Combine dataframes
all_mflow = pd.concat([mflow_orig, mflow_ncrs], ignore_index=True)
all_metad = pd.concat([metad_orig, metad_ncrs], ignore_index=True)

### Add HydroBASIN Data

In [193]:
# Load in attribute geospatial data (this takes a while)
gdf_basins = gpd.read_file(
    'assets/data/additional_sites/support_data/hydroBASINS_additional.gpkg')

# Convert coordinates to geometry
geometry = [Point(xy) for xy in zip(all_metad['longitude'], all_metad['latitude'])]
gdf_sites = gpd.GeoDataFrame(all_metad, geometry=geometry, crs=gdf_basins.crs)

# Select columns of interest
cols_int = ['inu_pc_smn','inu_pc_smx','inu_pc_slt','inu_pc_umn','inu_pc_umx',
            'inu_pc_ult','lka_pc_sse','lka_pc_use','dor_pc_pva','slp_dg_sav',
            'slp_dg_uav','sgr_dk_sav','tmp_dc_uyr','ari_ix_sav','ari_ix_uav',
            'cmi_ix_uyr','snw_pc_uyr','glc_pc_s01','glc_pc_s02','glc_pc_s03',
            'glc_pc_s04','glc_pc_s05','glc_pc_s06','glc_pc_s07','glc_pc_s08',
            'glc_pc_s09','glc_pc_s10','glc_pc_s11','glc_pc_s12','glc_pc_s13',
            'glc_pc_s14','glc_pc_s15','glc_pc_s16','glc_pc_s17','glc_pc_s18',
            'glc_pc_s19','glc_pc_s20','glc_pc_s21','glc_pc_s22','glc_pc_u01',
            'glc_pc_u02','glc_pc_u03','glc_pc_u04','glc_pc_u05','glc_pc_u06',
            'glc_pc_u07','glc_pc_u08','glc_pc_u09','glc_pc_u10','glc_pc_u11',
            'glc_pc_u12','glc_pc_u13','glc_pc_u14','glc_pc_u15','glc_pc_u16',
            'glc_pc_u17','glc_pc_u18','glc_pc_u19','glc_pc_u20','glc_pc_u21',
            'glc_pc_u22','wet_pc_sg1','wet_pc_sg2','wet_pc_ug1','wet_pc_ug2',
            'for_pc_sse','for_pc_use','crp_pc_sse','crp_pc_use','pst_pc_sse',
            'pst_pc_use','ire_pc_sse','ire_pc_use','gla_pc_sse','gla_pc_use',
            'prm_pc_sse','prm_pc_use','pac_pc_sse','pac_pc_use','cly_pc_sav',
            'cly_pc_uav','slt_pc_sav','slt_pc_uav','snd_pc_sav','snd_pc_uav',
            'soc_th_sav','soc_th_uav','swc_pc_syr','swc_pc_uyr','swc_pc_s01',
            'swc_pc_s02','swc_pc_s03','swc_pc_s04','swc_pc_s05','swc_pc_s06',
            'swc_pc_s07','swc_pc_s08','swc_pc_s09','swc_pc_s10','swc_pc_s11',
            'swc_pc_s12','kar_pc_sse','kar_pc_use','ero_kh_sav','ero_kh_uav',
            'ppd_pk_sav','ppd_pk_uav','urb_pc_sse','urb_pc_use','nli_ix_sav',
            'nli_ix_uav','rdd_mk_sav','rdd_mk_uav','hft_ix_s93','hft_ix_u93',
            'hft_ix_s09','hft_ix_u09','gwt_cm_sav','run_mm_syr','lkv_mc_usu',
            'rev_mc_usu','ria_ha_ssu','ria_ha_usu','riv_tc_ssu','riv_tc_usu',
            'pre_mm_uyr','pet_mm_syr','pet_mm_s01', 'pet_mm_s02','pet_mm_s03',
            'pet_mm_s04','pet_mm_s05','pet_mm_s06','pet_mm_s07','pet_mm_s08',
            'pet_mm_s09','pet_mm_s10','pet_mm_s11','pet_mm_s12','pet_mm_uyr',
            'aet_mm_syr','aet_mm_s01','aet_mm_s02','aet_mm_s03','aet_mm_s04',
            'aet_mm_s05','aet_mm_s06','aet_mm_s07','aet_mm_s08','aet_mm_s09',
            'aet_mm_s10','aet_mm_s11','aet_mm_s12','aet_mm_uyr','pop_ct_ssu',
            'pop_ct_usu','clz_cl_smj','cls_cl_smj','glc_cl_smj','pnv_cl_smj',
            'wet_cl_smj','tbi_cl_smj','tec_cl_smj','fmh_cl_smj','fec_cl_smj',
            'lit_cl_smj', 'SUB_AREA', 'UP_AREA','COAST','ORDER_','ENDO','geometry']

# Join relevant basin data
gdf_join = gpd.sjoin(gdf_sites, gdf_basins[cols_int], how='left', op='within')

# Code missing values as NaN
gdf_join = gdf_join.replace(-9999, np.nan)

# Replace points with basin polygon
polygons = gdf_join.merge(gdf_basins, how='left', left_on='index_right', right_index=True)
gdf_join.geometry = polygons.geometry_y

# Drop extra columns
cols_drop = ['nrcs_name', 'usgs_id', 'elevation', 'latitude', 'longitude',
             'season_start_month', 'season_end_month', 'geometry', 'index_right']
gdf_join = gdf_join.drop(cols_drop, axis=1)

# Convert categorical columns to strings
cols_cat = ['COAST', 'ORDER_', 'ENDO', 'clz_cl_smj', 'cls_cl_smj',
            'glc_cl_smj', 'pnv_cl_smj', 'wet_cl_smj', 'tbi_cl_smj',
            'tec_cl_smj', 'fmh_cl_smj', 'fec_cl_smj', 'lit_cl_smj']
gdf_join[cols_cat] = gdf_join[cols_cat].astype(str)

# Ensure data is geodataframe
gdf_join = gpd.GeoDataFrame(gdf_join, geometry=polygons.geometry_y, crs=gdf_basins.crs)

### Add SWANN Data

In [247]:
# Select years
START_YEAR = 1984
END_YEAR = 2024
YEARS = [y for y in range(START_YEAR, END_YEAR)]

# Select data
ROOT = 'https://climate.arizona.edu/data/UA_SWE/'
LOCATION = 'data/swann'

# Download data
os.makedirs(LOCATION, exist_ok=True)
for yr in tqdm(YEARS):
    fn = f'UA_SWE_Depth_WY{yr}.nc'
    url = ROOT + '/' + fn
    target = os.path.join(LOCATION, fn)
    with open(target, 'wb') as f:
        response = requests.get(url, verify=False)
        f.write(response.content)

swe_volumes = []
for yr in tqdm(YEARS):
    fn = os.path.join('data/swann', f'UA_SWE_Depth_WY{yr}.nc')
    try:
        ds = rioxarray.open_rasterio(fn, variable='SWE', mask_and_scale=True)
    except IOError:
        continue

    # Perform an initial clip to reduce the size of the array
    ds = ds.rio.write_crs(4326)
    ds = ds.rio.reproject("EPSG:4326")

    ds['SWE'] /= 1000  # mm -> m
    ds = ds.fillna(0)
    ds = ds.rio.write_crs(4326)
    ds_clipped_rgn = ds.rio.clip(gdf_join.geometry.values)

    for i in range(0, len(gdf_join)):
        try:
            # Clip to current polygon
            ds_clipped_poly = ds_clipped_rgn.rio.clip([gdf_join.iloc[i].geometry])

            # Set -999 to nan then convert to dataframe
            df = ds_clipped_poly.to_dataframe()
            df = df.dropna()
            df = df.reset_index()

            # Group by time and average over grid cells
            swe_vol = df.groupby(['time'])['SWE'].mean()

            # Format dataframe
            swe_vol = pd.DataFrame(swe_vol).reset_index()
            swe_vol['site_id'] = gdf_join.iloc[i].site_id
            swe_vol = swe_vol.set_index(['site_id', 'time'])
            swe_vol = swe_vol.rename({'SWE': 'SWE_depth_m'}, axis=1)
            swe_volumes.append(swe_vol)
        except Exception as e:
            print(f"No data found for: {gdf_join.iloc[i]['site_id']}")
            continue

Concatenate dataframes and write output
swe_volumes = pd.concat(swe_volumes, axis=0)
swe_volumes.to_csv("assets/data/additional_sites/swann_swe_extra.csv")

# Filter to years of interest
swe_volumes_week = swe_volumes.copy().reset_index()
swe_volumes_week['date'] = pd.to_datetime(
    swe_volumes_week['time'].astype(str)).dt.strftime('%Y-%m-%d')
swe_volumes_week = swe_volumes_week[pd.to_datetime(
    swe_volumes_week['date']) >= pd.Timestamp("1985-01-01")]

# Define start and end dates
start_date = datetime.date(1984, 1, 1)
end_date = datetime.date(2024, 1, 1)

# Initialize an empty list to store weeks
week_list = []

# Generate weeks between start_date and end_date
current_date = start_date
while current_date < end_date:
    for day in [1, 8, 15, 22]:
        week = current_date + datetime.timedelta(days=(day - current_date.weekday() - 1))
        if week < end_date:
            week_list.append(week.strftime('%Y-%m-%d'))
    current_date += datetime.timedelta(days=7)
    
# Function to round down the day to the nearest value less than or equal to the day
def round_day_down(date):
    day = date.day
    nearest_values = [1, 8, 15, 22]

    # Find the nearest value less than or equal to the day
    rounded_day = max(filter(lambda x: x <= day, nearest_values))
    return date.replace(day=rounded_day)

 # Create a new column 'Rounded_Day_Column' based on 'Date_Column'
swe_volumes_week['week_start_date'] = pd.to_datetime(
    swe_volumes_week['date']).apply(round_day_down)
swe_volumes_week = swe_volumes_week.drop(['date', 'time'], axis=1)

# Aggregate by week
swe_volumes_week = swe_volumes_week.groupby(
    ['site_id','week_start_date']).mean().reset_index()

SWE_depth_m
site_id                       time                            
hungry_horse_reservoir_inflow 2021-10-01 00:00:00     0.000029
                              2021-10-02 00:00:00     0.000007
                              2021-10-03 00:00:00     0.000000
                              2021-10-04 00:00:00     0.000000
                              2021-10-05 00:00:00     0.000000
...                                                        ...
09386950:NM:USGS              2022-09-26 00:00:00     0.000000
                              2022-09-27 00:00:00     0.000000
                              2022-09-28 00:00:00     0.000000
                              2022-09-29 00:00:00     0.000000
                              2022-09-30 00:00:00     0.000000

[219000 rows x 1 columns]

### Add USGS Streamflow Data

In [172]:
# Replace missing nrcs_name
all_metad['nrcs_name'] = np.where(
    all_metad['nrcs_name'].isna(), all_metad['site_id'], all_metad['nrcs_name'])

# Create empty list
usbr_all = []

# Loop through years
for year in range(1985, 2025):
    print(f"#######Retreiving data for {year}#######")

    # Loop through forecasting sites
    site_list = [item.lower().replace(' ', '_') for item in all_metad['nrcs_name']]
    for site in site_list:
        file_path = f"./assets/data/additional_sites/support_data/usgs_streamflow/FY{year}/{site}.csv"

        #  # Check if the file exists for the station
        if not os.path.exists(file_path):
            print(f"File not found for site {site}. Skipping...")
            continue

        # Read in daily flow data for the selected year
        flow_data = pd.read_csv(file_path)
        flow_data['site_id'] = site

        # Select data of interest
        flow_data = flow_data[['site_id', 'datetime', '00060_Mean']]

        # Append to list
        usbr_all.append(flow_data)

# Combine data
result = pd.concat(usbr_all, axis=0, ignore_index=True)

# Perform additional cleaning
result['datetime'] = pd.to_datetime(result['datetime']).dt.strftime('%Y-%m-%d')
result['00060_Mean'] = np.where(result['00060_Mean'] == -999999, np.nan, result['00060_Mean'])

# Filter to years of interest
result_week = result.copy()
result_week['date'] = pd.to_datetime(result['datetime']).dt.date
result_week = result_week[pd.to_datetime(
    result_week['date']) >= pd.Timestamp("1984-01-01")]

# Define start and end dates
start_date = datetime.date(1984, 1, 1)
end_date = datetime.date(2024, 1, 1)

# Initialize an empty list to store weeks
week_list = []

# Generate weeks between start_date and end_date
current_date = start_date
while current_date < end_date:
    for day in [1, 8, 15, 22]:
        week = current_date + datetime.timedelta(days=(day - current_date.weekday() - 1))
        if week < end_date:
            week_list.append(week.strftime('%Y-%m-%d'))
    current_date += datetime.timedelta(days=7)
    
# Function to round down the day to the nearest value less than or equal to the day
def round_day_down(date):
    day = date.day
    nearest_values = [1, 8, 15, 22]

    # Find the nearest value less than or equal to the day
    rounded_day = max(filter(lambda x: x <= day, nearest_values))
    return date.replace(day=rounded_day)

 # Create a new column 'Rounded_Day_Column' based on 'Date_Column'
result_week['week_start_date'] = result_week['date'].apply(round_day_down)
result_week = result_week.drop(['date', 'datetime'], axis=1)

# Aggregate by week
result_week = result_week.groupby(
    ['site_id','week_start_date']).mean().reset_index()

# Export data
streamflow['site_id'] = streamflow['site_id'].str.replace(',', '')
result_week.to_csv('./assets/data/additional_sites/usgs_streamflow.csv', index=False)

#######Retreiving data for 1985#######
File not found for site ruedi_reservoir_inflow. Skipping...
File not found for site fontenelle_reservoir_inflow. Skipping...
File not found for site american_river_folsom_lake. Skipping...
File not found for site skagit_ross_reservoir. Skipping...
File not found for site alamosa_ck_ab_terrace_reservoir. Skipping...
File not found for site arkansas_r_at_salida. Skipping...
File not found for site asotin_ck_at_asotin. Skipping...
File not found for site beaverhead_r_nr_grant. Skipping...
File not found for site big_c_bl_huntington_lk_ca. Skipping...
File not found for site big_ck_nr_randolph. Skipping...
File not found for site big_goose_ck_nr_sheridan. Skipping...
File not found for site big_hole_r_bl_big_lake_ck_at_wisdom. Skipping...
File not found for site bitterroot_r_nr_missoula. Skipping...
File not found for site boca_res_local_inflow. Skipping...
File not found for site bowl_canyon_ck_ab_asaayi_lake. Skipping...
File not found for site brad

File not found for site bitterroot_r_nr_missoula. Skipping...
File not found for site boca_res_local_inflow. Skipping...
File not found for site bowl_canyon_ck_ab_asaayi_lake. Skipping...
File not found for site bradley_lake_inflow. Skipping...
File not found for site bumping_lake_inflow. Skipping...
File not found for site captain_tom_wash_nr_two_gray_hills. Skipping...
File not found for site chalk_ck_nr_nathrop. Skipping...
File not found for site city_ck_nr_slc. Skipping...
File not found for site clark_fork_r_ab_milltown. Skipping...
File not found for site clark_fork_r_bl_cabinet_gorge_dam. Skipping...
File not found for site cle_elum_lake_inflow. Skipping...
File not found for site clear_lake_inflow. Skipping...
File not found for site colorado_r_at_glen_canyon_dam. Skipping...
File not found for site colorado_r_bl_lake_granby. Skipping...
File not found for site costilla_ck_bl_costilla_dam. Skipping...
File not found for site crane_prairie_reservoir_inflow. Skipping...
File not

File not found for site great_salt_lake_inflow. Skipping...
File not found for site green_peter_lake_inflow. Skipping...
File not found for site green_r_at_flaming_gorge_dam. Skipping...
File not found for site gulkana_r_at_sourdough. Skipping...
File not found for site gunnison_r_at_blue_mesa_dam. Skipping...
File not found for site hams_fk_bl_viva_naughton_reservoir. Skipping...
File not found for site hellroaring_creek_ab_reservoir_nr_polson. Skipping...
File not found for site huerfano_r_nr_redwing. Skipping...
File not found for site huntington_ck_bl_electric_lake. Skipping...
File not found for site hyalite_reservoir_inflow. Skipping...
File not found for site icicle_ck_nr_leavenworth. Skipping...
File not found for site jarbidge_river_below_jarbidge. Skipping...
File not found for site jefferson_r_nr_twin_bridges. Skipping...
File not found for site kachess_reservoir_inflow. Skipping...
File not found for site keechelus_reservoir_inflow. Skipping...
File not found for site klick

File not found for site muddy_ck_bl_paonia_reservoir. Skipping...
File not found for site muddy_creek_bl_wolford_mtn_reservoir. Skipping...
File not found for site naches_r_nr_naches. Skipping...
File not found for site nf_humboldt_r_at_devils_gate. Skipping...
File not found for site nf_shoshone_r_at_wapiti. Skipping...
File not found for site north_platte_r_bl_glendo_reservoir. Skipping...
File not found for site north_platte_r_bl_seminoe_reservoir. Skipping...
File not found for site oakley_reservoir_inflow. Skipping...
File not found for site ochoco_reservoir_inflow. Skipping...
File not found for site oroville_reservoir_inflow. Skipping...
File not found for site owens_r_bl_tinemaha_re_nr_big_pine_ca. Skipping...
File not found for site parleys_ck_nr_slc. Skipping...
File not found for site pineview_reservoir_inflow. Skipping...
File not found for site powder_r_nr_sumpter. Skipping...
File not found for site price_r_nr_scofield_reservoir. Skipping...
File not found for site priest

File not found for site salt_ck_at_nephi. Skipping...
File not found for site san_juan_r_nr_archuleta. Skipping...
File not found for site sangre_de_cristo_ck. Skipping...
File not found for site seely_ck_bl_joes_valley_resv. Skipping...
File not found for site sf_flathead_r_nr_hungry_horse. Skipping...
File not found for site slate_r_nr_crested_butte. Skipping...
File not found for site smith_r_bl_eagle_ck. Skipping...
File not found for site snake_r_bl_lower_granite_dam-nws. Skipping...
File not found for site south_platte_r_ab_spinney_res. Skipping...
File not found for site south_platte_r_at_south_platte. Skipping...
File not found for site south_platte_r_bl_antero_reservoir. Skipping...
File not found for site south_platte_r_bl_cheesman_lk. Skipping...
File not found for site south_platte_r_nr_lake_george. Skipping...
File not found for site stampede_reservoir_inflow. Skipping...
File not found for site swift_reservoir_inflow. Skipping...
File not found for site taiya_r_nr_skagway

...

File not found for site strawberry_ck_nr_prairie_city. Skipping...
File not found for site strawberry_r_nr_soldier_springs. Skipping...
File not found for site swift_reservoir_inflow. Skipping...
File not found for site teanaway_r_bl_forks_nr_cle_elum. Skipping...
File not found for site tobacco_r_nr_eureka. Skipping...
File not found for site trinchera_ck_ab_turners_ranch. Skipping...
File not found for site trout_ck_nr_denio. Skipping...
File not found for site truckee_r_ab_farad_sidewater. Skipping...
File not found for site tule_r_bl_success_dam_ca. Skipping...
File not found for site twentymile_ck_nr_adel. Skipping...
File not found for site umatilla_r_at_pendleton. Skipping...
File not found for site upper_klamath_lake_inflow. Skipping...
File not found for site upper_williamson_r. Skipping...
File not found for site utah_lake_inflow. Skipping...
File not found for site ute_ck_nr_fort_garland. Skipping...
File not found for site wf_duchesne_r_at_vat_diversion. Skipping...
File no

File not found for site mill_ck_nr_slc. Skipping...
File not found for site millerton_lake_at_friant,_ca. Skipping...
File not found for site muddy_ck_bl_paonia_reservoir. Skipping...
File not found for site naches_r_nr_naches. Skipping...
File not found for site navajo_r_bl_oso_diversion_dam_nr_chromo. Skipping...
File not found for site nf_malheur_r_at_beulah. Skipping...
File not found for site north_platte_r_bl_glendo_reservoir. Skipping...
File not found for site north_platte_r_bl_guernsey_reservoir. Skipping...
File not found for site north_platte_r_bl_seminoe_reservoir. Skipping...
File not found for site oakley_reservoir_inflow. Skipping...
File not found for site ochoco_reservoir_inflow. Skipping...
File not found for site oroville_reservoir_inflow. Skipping...
File not found for site owens_r_bl_tinemaha_re_nr_big_pine_ca. Skipping...
File not found for site parleys_ck_nr_slc. Skipping...
File not found for site pine_ck_nr_oxbow. Skipping...
File not found for site pineview_re

File not found for site silvies_r_nr_burns. Skipping...
File not found for site slate_r_nr_crested_butte. Skipping...
File not found for site snake_r_bl_lower_granite_dam-nws. Skipping...
File not found for site south_boulder_ck_nr_eldorado_springs. Skipping...
File not found for site south_platte_r_ab_spinney_res. Skipping...
File not found for site south_platte_r_at_south_platte. Skipping...
File not found for site south_platte_r_bl_antero_reservoir. Skipping...
File not found for site south_platte_r_bl_cheesman_lk. Skipping...
File not found for site south_platte_r_nr_lake_george. Skipping...
File not found for site st._vrain_ck_at_lyons. Skipping...
File not found for site stampede_reservoir_inflow. Skipping...
File not found for site strawberry_ck_nr_prairie_city. Skipping...
File not found for site strawberry_r_nr_soldier_springs. Skipping...
File not found for site swift_reservoir_inflow. Skipping...
File not found for site teanaway_r_bl_forks_nr_cle_elum. Skipping...
File not f

File not found for site skagit_ross_reservoir. Skipping...
File not found for site boysen_reservoir_inflow. Skipping...
File not found for site alamosa_ck_ab_terrace_reservoir. Skipping...
File not found for site arkansas_r_at_salida. Skipping...
File not found for site baker_r_at_concrete. Skipping...
File not found for site bear_ck_nr_wallowa. Skipping...
File not found for site bear_r_bl_stewart_dam. Skipping...
File not found for site beaverhead_r_nr_grant. Skipping...
File not found for site big_c_bl_huntington_lk_ca. Skipping...
File not found for site big_cottonwood_ck_nr_slc. Skipping...
File not found for site big_goose_ck_nr_sheridan. Skipping...
File not found for site big_thompson_r_at_canyon_mouth. Skipping...
File not found for site big_wood_r_ab_magic_reservoir. Skipping...
File not found for site bitterroot_r_nr_missoula. Skipping...
File not found for site boca_res_local_inflow. Skipping...
File not found for site boulder_ck_nr_orodell. Skipping...
File not found for s

File not found for site rock_ck_nr_buffalo. Skipping...
File not found for site saguache_ck_nr_saguache. Skipping...
File not found for site san_juan_r_nr_archuleta. Skipping...
File not found for site sangre_de_cristo_ck. Skipping...
File not found for site seely_ck_bl_joes_valley_resv. Skipping...
File not found for site sf_rio_grande_at_south_fork. Skipping...
File not found for site sf_san_joaquin_r_l_hooper_c_nr_florence. Skipping...
File not found for site sf_walla_walla_r_nr_milton-freewater. Skipping...
File not found for site shell_ck_nr_shell. Skipping...
File not found for site silver_c_bl_camino_div_dam_ca. Skipping...
File not found for site silvies_r_nr_burns. Skipping...
File not found for site slate_r_nr_crested_butte. Skipping...
File not found for site snake_r_bl_lower_granite_dam-nws. Skipping...
File not found for site south_boulder_ck_nr_eldorado_springs. Skipping...
File not found for site south_crow_ck_nr_ronan. Skipping...
File not found for site south_platte_r_

### Combine Data and Export

In [218]:
# Link area to snow volume data
swe_volumes_area = swe_volumes_week.merge(
    gdf_join[['site_id','SUB_AREA']], how='left', on='site_id')

# Scale snow volume by area
swe_volumes_area['SWE_depth_m_AREA_SCALED'] = swe_volumes_area[
    'SWE_depth_m'] * swe_volumes_area['SUB_AREA']

# Drop area column
swe_volumes_area = swe_volumes_area.drop('SUB_AREA', axis=1)

# Export data
all_metad.to_csv("assets/data/additional_sites/metadata.csv", index=False)
swe_volumes_area.to_csv("assets/data/additional_sites/swann_swe.csv", index=False)
all_mflow.to_csv("assets/data/additional_sites/train_monthly_naturalized_flow.csv", index=False)
gdf_join.to_csv("assets/data/additional_sites/hydrobasins_summary.csv", index=False)

In [8]:
# Transform into delta models

In [190]:
all_mflow[all_mflow['site_id'] == 'american_river_folsom_lake']

,site_id,forecast_year,year,month,volume
